In [3]:
import numpy as np 
import pandas as pd 
import altair as alt 
import os 
import re 
import ntpath
from Commons.data_processing import *
from Commons.DataProcessors.pd_processor import PDProcessor


In [4]:
files = get_files(r'E:\Research\Proteome Discoverer\20210825_StdProt_Glyco')

In [7]:

read_data = None

for file in files:
    base_name, _ = ntpath.splitext(ntpath.basename(file))
    div = base_name.split('-')[0]
    div = div.split('_')
    conc, temp, run = div[-3:]
    
    data = PDProcessor([file], sample_name='_'.join(div[-3:]))
    data.add_special_column('concentration', conc)
    data.add_special_column('temperature', temp)
    data.add_special_column('run', run)

    if read_data is None:
        read_data = data 
    else:
        read_data.join_processors(data)

    
read_data.alias_engine('')        

In [11]:
peptides = read_data.peptides[~read_data.peptides.isna()]

In [18]:
cutoffs = [50, 100, 150, 200, 250, 300, 350, 400, 500]
nums = []
for cut in cutoffs:
    nums.append(len(peptides[peptides.byonic_score >= cut]))

res = pd.DataFrame({
    'cutoffs':cutoffs,
    'vals':nums
})

alt.Chart(res).mark_bar().encode(
    x='cutoffs:N',
    y='vals:Q'
)

alt.Chart(...)